In [3]:
from google.colab import output
output.disable_custom_widget_manager()

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding

In [5]:
raw_datasets = load_dataset('glue','mrpc')
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example['sentence1'],example['sentence2'],truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function,batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments('test-trainer')
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    processing_class=tokenizer,  #
)
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ldq13669343985 (ldq13669343985-ncu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.595300
1000,0.394600


TrainOutput(global_step=1377, training_loss=0.4374721031216668, metrics={'train_runtime': 252.3571, 'train_samples_per_second': 43.605, 'train_steps_per_second': 5.457, 'total_flos': 405114969714960.0, 'train_loss': 0.4374721031216668, 'epoch': 3.0})

**Evaluation**

In [8]:
predictions = trainer.predict(tokenized_datasets['validation'])

In [9]:
print(predictions.predictions.shape,predictions.label_ids.shape)

(408, 2) (408,)


In [11]:
print(type(predictions))
print(type(predictions.predictions))
print(type(predictions.label_ids))

<class 'transformers.trainer_utils.PredictionOutput'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [12]:
import numpy as np
preds = np.argmax(predictions.predictions,axis=-1)

In [16]:
!pip install  evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00


In [17]:
import evaluate

metric = evaluate.load('glue','mrpc')
metric.compute(predictions=preds,references=predictions.label_ids)

{'accuracy': 0.8480392156862745, 'f1': 0.8945578231292517}

In [18]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.428027,0.794118,0.867508
2,0.529600,0.517542,0.848039,0.894558
3,0.266700,0.790612,0.840686,0.890756


TrainOutput(global_step=1377, training_loss=0.3270585469430828, metrics={'train_runtime': 236.1828, 'train_samples_per_second': 46.591, 'train_steps_per_second': 5.83, 'total_flos': 405114969714960.0, 'train_loss': 0.3270585469430828, 'epoch': 3.0})